## 用requests抓取链家网信息

![](./images/lianjia.png)

来动手做一个练习，做到学以致用。<br>
这次，我们来爬取链家网的一些内容，用的工具依旧是大家熟悉的requests和BeautifulSoup

### 1.准备工作

编写爬虫前的准备工作，我们需要导入用到的库，这里主要使用的是requests和BeautifulSoup两个。还有一个Time库，负责设置每次抓取的休息时间。

In [1]:
import requests
import time
from bs4 import BeautifulSoup

### 2.抓取列表页

开始抓取前当然应该了解一下目标网站URL结构咯。<br>
链家网的二手房列表页面共有100个，URL结构为http://bj.lianjia.com/ershoufang/pg9/ 
其中

- bj表示城市
- /ershoufang/是频道名称
- pg9是页面码。

我们要抓取的是北京的二手房频道，所以前面的部分不会变，属于固定部分，后面的页面码需要在1-100间变化，属于可变部分。将URL分为两部分，前面的固定部分赋值给url，后面的可变部分使用for循环遍历页面。

In [2]:
#设置列表页URL的固定部分
url='http://bj.lianjia.com/ershoufang/'
#设置页面页的可变部分
page=('pg')

这里提一个小小的醒，我们最好在http请求中设置一个头部信息，否则很容易被封ip。头部信息网上有很多现成的，也可以使用httpwatch等工具来查看。

In [3]:
#设置请求头部信息
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
'Accept':'text/html;q=0.9,*/*;q=0.8',
'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
'Accept-Encoding':'gzip',
'Connection':'close',
'Referer':'http://www.baidu.com/link?url=_andhfsjjjKRgEWkj7i9cFmYYGsisrnm2A-TN3XZDQXxvGsM9k9ZZSnikW2Yds4s&amp;amp;wd=&amp;amp;eqid=c3435a7d00146bd600000003582bfd1f'
}

我们使用for循环生成1-100的数字，转化格式后与前面的URL固定部分拼成要抓取的URL。这里我们设置每两个页面间隔0.5秒。抓取到的页面保存在html中。

In [4]:
#循环抓取列表页信息
for i in range(1,10):
     if i == 1:
          i=str(i)
          a=(url+page+i+'/')
          r=requests.get(url=a,headers=headers)
          html=r.content
     else:
          i=str(i)
          a=(url+page+i+'/')
          r=requests.get(url=a,headers=headers)
          html2=r.content
          html = html + html2
     #每次间隔3秒
     time.sleep(3)

### 页面解析
页面抓取的工作算是完成了，内容在html中，下一步就要进行页面解析了。我们依旧使用BeautifulSoup对页面进行解析。

In [5]:
#解析抓取的页面内容
lj=BeautifulSoup(html,'html.parser')

完成页面解析后就可以对页面中的关键信息进行提取了。下面我们分别对房源的总价，房源信息和关注度三部分进行提取。
把页面div标签中class=priceInfo的部分提取出来，并使用for循环将其中每个房源的总价数据存在tp中。

In [7]:
#提取房源总价
price=lj.find_all('div','priceInfo')
tp=[]
for a in price:
    totalPrice=a.span.string
    tp.append(totalPrice)

来看看爬下来的**房价数据**
![](./images/house-price.jpg)

In [8]:
tp

[u'1515',
 u'435',
 u'695',
 u'1850',
 u'330',
 u'760',
 u'650',
 u'184',
 u'490',
 u'468',
 u'425',
 u'1030',
 u'352',
 u'710',
 u'518',
 u'405',
 u'665',
 u'660',
 u'540',
 u'735',
 u'542',
 u'730',
 u'370',
 u'750',
 u'1800',
 u'810',
 u'200',
 u'1170',
 u'945',
 u'1850']

提取房源信息和关注度的方法与提取房源价格的方法类似，下面是具体的代码，房源信息存储在hi中，关注度存储在fi中。

In [9]:
#提取房源信息
houseInfo=lj.find_all('div',attrs={'class':'houseInfo'})
 
hi=[]
for b in houseInfo:
    house=b.get_text()
    hi.append(house)

来看看**房源信息**
![](./images/house-info.jpg)

In [11]:
for item in hi:
    print item

首开知语城  | 3室2厅 | 151.39平米 | 南 北 | 精装
疃里新村  | 3室1厅 | 104.96平米 | 南 北 | 简装
天时名苑  | 2室2厅 | 121.3平米 | 南 北 | 精装 | 有电梯
恋日嘉园一期  | 5室2厅 | 283.1平米 | 南 北 | 精装
美林湾  | 2室2厅 | 91.26平米 | 南 北 | 精装
郁花园二里  | 4室2厅 | 182.5平米 | 南 北 | 简装
雅成一里  | 3室2厅 | 104.56平米 | 南 北 | 简装
福城上上城五期  | 1室1厅 | 57.43平米 | 南 | 简装
云趣园一区  | 3室2厅 | 103.34平米 | 南 北 | 简装
天赐良园一期  | 2室1厅 | 74.32平米 | 南 北 | 简装
11站  | 1室1厅 | 69.12平米 | 南 | 精装 | 有电梯
流星花园三区  | 3室2厅 | 151.42平米 | 南 北 | 精装 | 有电梯
住欣家园  | 2室1厅 | 70平米 | 东 南 | 简装 | 有电梯
御景山  | 2室2厅 | 114.52平米 | 南 北 | 其他
新城阳光  | 2室1厅 | 94.43平米 | 南 | 精装 | 有电梯
武圣西里  | 2室1厅 | 57.83平米 | 南 北 | 简装
珠江骏景中区  | 2室1厅 | 97.93平米 | 南 北 | 毛坯
莲香园  | 2室1厅 | 82.11平米 | 东南 | 简装 | 有电梯
天通苑本三区  | 3室2厅 | 128.66平米 | 南 北 | 精装
厂洼小区  | 3室1厅 | 69.3平米 | 南 北 | 简装
八角南里  | 2室1厅 | 81.22平米 | 南 北 | 简装
天通苑中苑  | 3室2厅 | 153.1平米 | 南 北 | 精装 | 有电梯
裕龙一区  | 2室1厅 | 84.33平米 | 南 北 | 精装
天通苑东三区  | 4室2厅 | 207.11平米 | 南 北 | 简装
立恒名苑  | 5室2厅 | 312.54平米 | 西南 | 简装
香林郡  | 4室2厅 | 155.2平米 | 南 北 | 简装
福城上上城五期  | 1室1厅 | 69.07平米 | 南 | 简装
金隅丽港城  | 3室2厅 | 132.52平米 | 东

In [13]:
#提取房源关注度
followInfo=lj.find_all('div',attrs={'class':'followInfo'})
 
fi=[]
for c in followInfo:
    follow=c.get_text()
    fi.append(follow)

再来看看关注度状况
![](./images/house-followed.jpg)

In [14]:
for item in fi:
    print item

283人关注 / 共38次带看 / 19天以前发布
159人关注 / 共28次带看 / 6天以前发布
538人关注 / 共64次带看 / 17天以前发布
135人关注 / 共35次带看 / 28天以前发布
68人关注 / 共39次带看 / 6天以前发布
178人关注 / 共53次带看 / 27天以前发布
217人关注 / 共24次带看 / 13天以前发布
113人关注 / 共42次带看 / 14天以前发布
307人关注 / 共85次带看 / 18天以前发布
271人关注 / 共51次带看 / 15天以前发布
80人关注 / 共30次带看 / 12天以前发布
99人关注 / 共45次带看 / 28天以前发布
303人关注 / 共31次带看 / 11天以前发布
29人关注 / 共51次带看 / 11天以前发布
122人关注 / 共37次带看 / 14天以前发布
102人关注 / 共28次带看 / 6天以前发布
100人关注 / 共48次带看 / 12天以前发布
75人关注 / 共65次带看 / 12天以前发布
57人关注 / 共35次带看 / 12天以前发布
105人关注 / 共31次带看 / 12天以前发布
95人关注 / 共49次带看 / 15天以前发布
67人关注 / 共55次带看 / 14天以前发布
69人关注 / 共21次带看 / 7天以前发布
516人关注 / 共110次带看 / 4个月以前发布
41人关注 / 共23次带看 / 23天以前发布
72人关注 / 共23次带看 / 23天以前发布
199人关注 / 共76次带看 / 29天以前发布
59人关注 / 共46次带看 / 29天以前发布
113人关注 / 共38次带看 / 26天以前发布
226人关注 / 共42次带看 / 1个月以前发布


### 清洗数据并整理到数据表中

我们将之前爬取到的信息进行汇总，并导入pandas之中生成数据表。便于后面的分析。

In [15]:
#导入pandas库
import pandas as pd
#创建数据表
house=pd.DataFrame({'totalprice':tp,'houseinfo':hi,'followinfo':fi})
#查看数据表的内容
house.head()

followinfo                                houseinfo  \
0  283人关注 / 共38次带看 / 19天以前发布      首开知语城  | 3室2厅 | 151.39平米 | 南 北 | 精装   
1   159人关注 / 共28次带看 / 6天以前发布       疃里新村  | 3室1厅 | 104.96平米 | 南 北 | 简装   
2  538人关注 / 共64次带看 / 17天以前发布  天时名苑  | 2室2厅 | 121.3平米 | 南 北 | 精装 | 有电梯   
3  135人关注 / 共35次带看 / 28天以前发布      恋日嘉园一期  | 5室2厅 | 283.1平米 | 南 北 | 精装   
4    68人关注 / 共39次带看 / 6天以前发布         美林湾  | 2室2厅 | 91.26平米 | 南 北 | 精装   

  totalprice  
0       1515  
1        435  
2        695  
3       1850  
4        330

很尴尬的是，大家看得到，很多信息是糊在一块的，不能直接使用，所以咱们再做一些数据提取和清洗的工作。如房源信息，在表中每个房源的小区名称，户型，面积，朝向等信息都在一个字段中，无法直接使用。需要先进行分列操作。这里的规则比较明显，每个信息间都是以竖线分割的，因此我们只需要以竖线进行分列即可。

In [16]:
#对房源信息进行分列
houseinfo_split = pd.DataFrame((x.split('|') for x in house.houseinfo),index=house.index,columns=['xiaoqu','huxing','mianji','chaoxiang','zhuangxiu','dianti'])

现在再来看看我们整理好的数据

In [17]:
#查看分列结果
houseinfo_split.head()

xiaoqu  huxing      mianji chaoxiang zhuangxiu dianti
0   首开知语城     3室2厅    151.39平米       南 北         精装   None
1    疃里新村     3室1厅    104.96平米       南 北         简装   None
2    天时名苑     2室2厅     121.3平米       南 北        精装     有电梯
3  恋日嘉园一期     5室2厅     283.1平米       南 北         精装   None
4     美林湾     2室2厅     91.26平米       南 北         精装   None

把拆分后的数据拼接回原始数据中

In [18]:
#将分列结果拼接回原数据表
house=pd.merge(house,houseinfo_split,right_index=True, left_index=True)

In [19]:
house.head()

followinfo                                houseinfo  \
0  283人关注 / 共38次带看 / 19天以前发布      首开知语城  | 3室2厅 | 151.39平米 | 南 北 | 精装   
1   159人关注 / 共28次带看 / 6天以前发布       疃里新村  | 3室1厅 | 104.96平米 | 南 北 | 简装   
2  538人关注 / 共64次带看 / 17天以前发布  天时名苑  | 2室2厅 | 121.3平米 | 南 北 | 精装 | 有电梯   
3  135人关注 / 共35次带看 / 28天以前发布      恋日嘉园一期  | 5室2厅 | 283.1平米 | 南 北 | 精装   
4    68人关注 / 共39次带看 / 6天以前发布         美林湾  | 2室2厅 | 91.26平米 | 南 北 | 精装   

  totalprice    xiaoqu  huxing      mianji chaoxiang zhuangxiu dianti  
0       1515   首开知语城     3室2厅    151.39平米       南 北         精装   None  
1        435    疃里新村     3室1厅    104.96平米       南 北         简装   None  
2        695    天时名苑     2室2厅     121.3平米       南 北        精装     有电梯  
3       1850  恋日嘉园一期     5室2厅     283.1平米       南 北         精装   None  
4        330     美林湾     2室2厅     91.26平米       南 北         精装   None

使用相同的方法对房源关注度字段进行分列和拼接操作。这里的分列规则是斜杠。

In [20]:
#对房源关注度进行分列
followinfo_split = pd.DataFrame((x.split('/') for x in house.followinfo),index=house.index,columns=['guanzhu','daikan','fabu'])
#将分列后的关注度信息拼接回原数据表
house=pd.merge(house,followinfo_split,right_index=True, left_index=True)

In [21]:
house.head()

followinfo                                houseinfo  \
0  283人关注 / 共38次带看 / 19天以前发布      首开知语城  | 3室2厅 | 151.39平米 | 南 北 | 精装   
1   159人关注 / 共28次带看 / 6天以前发布       疃里新村  | 3室1厅 | 104.96平米 | 南 北 | 简装   
2  538人关注 / 共64次带看 / 17天以前发布  天时名苑  | 2室2厅 | 121.3平米 | 南 北 | 精装 | 有电梯   
3  135人关注 / 共35次带看 / 28天以前发布      恋日嘉园一期  | 5室2厅 | 283.1平米 | 南 北 | 精装   
4    68人关注 / 共39次带看 / 6天以前发布         美林湾  | 2室2厅 | 91.26平米 | 南 北 | 精装   

  totalprice    xiaoqu  huxing      mianji chaoxiang zhuangxiu dianti  \
0       1515   首开知语城     3室2厅    151.39平米       南 北         精装   None   
1        435    疃里新村     3室1厅    104.96平米       南 北         简装   None   
2        695    天时名苑     2室2厅     121.3平米       南 北        精装     有电梯   
3       1850  恋日嘉园一期     5室2厅     283.1平米       南 北         精装   None   
4        330     美林湾     2室2厅     91.26平米       南 北         精装   None   

   guanzhu    daikan      fabu  
0  283人关注    共38次带看    19天以前发布  
1  159人关注    共28次带看     6天以前发布  
2  538人关注    共64次带看    17天以前发布  
3  135人关注    共35次带看    28天以前发布  
4   68人关注    共39次带看     6天以前发布

好了，这就是咱们对链家网的部分数据爬取过程了。<br>
跃跃欲试的宝宝们何不自己动手，再解析一些内容试试。<br>
另外对**数据分析和可视化**感兴趣的同学，千万不要错过咱们后面的案例哦，会有后续的跟进数据分析和展示~